In [1]:
import pandas as pd
import numpy as np

In [4]:
steel=pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/00198/Faults.NNA',header=None)

In [5]:
steel.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


In [6]:
steel1=pd.read_csv('Steel Plate Faults.csv')

In [7]:
steel1.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Fault
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,Pastry
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,Pastry
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,Pastry
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,Pastry
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,Pastry


# Naive Bayesian

In [17]:
from sklearn import naive_bayes
from sklearn import preprocessing

In [18]:
l=[]
for i in range(len(steel1)):
    if steel1['TypeOfSteel_A3FalseFalse'][i]:
        l.append('A3')
    else:
        l.append('A4')

In [21]:
steel1['TypeOfSteel']=l

In [22]:
steel1['TypeOfSteel']=steel1['TypeOfSteel'].astype('category')

In [23]:
le = preprocessing.LabelEncoder()

In [24]:
l_en=le.fit_transform(l)

In [26]:
steel1['TypeOfSteel']=l_en

In [27]:
steel1['TypeOfSteel']=steel1['TypeOfSteel'].astype('category')

In [29]:
faults=steel1['Fault']

In [30]:
faults_en=le.fit_transform(faults)

In [31]:
steel1['Fault']=faults_en

In [33]:
steel1['Fault']=steel1['Fault'].astype('category')

In [34]:
steel1_num_normalized = preprocessing.scale(X = steel1.drop(['TypeOfSteel_A3FalseFalse','TypeOfSteel_A4FalseFalse','TypeOfSteel','Fault'],axis=1))

In [35]:
nb_classifier_multinom = naive_bayes.MultinomialNB()

In [48]:
np.atleast_2d(steel1['TypeOfSteel']).transpose().shape

(1941, 1)

In [47]:
nb_classifier_multinom.fit(X = np.atleast_2d(steel1['TypeOfSteel']).transpose(), y = steel1['Fault'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
df_steel1_num_normalized=pd.DataFrame(steel1_num_normalized)

In [50]:
nb_classifier = naive_bayes.GaussianNB()
nb_classifier.fit(X = df_steel1_num_normalized, y = steel1['Fault'])

GaussianNB(priors=None)

In [51]:
Cat_pred = nb_classifier_multinom.predict_proba(X = np.atleast_2d(steel1['TypeOfSteel']).transpose())

In [52]:
Num_pred=nb_classifier.predict_proba(X=df_steel1_num_normalized)

In [53]:
steel1_probs = np.column_stack((Cat_pred, Num_pred))

In [55]:
nb_classifier.fit(X = steel1_probs,y = steel1['Fault'])

GaussianNB(priors=None)

In [57]:
steel1_final_pred = nb_classifier.predict(X = steel1_probs)

In [58]:
from sklearn import metrics
metrics.confusion_matrix(steel1['Fault'],steel1_final_pred)

array([[  1, 366,  22,   5,   4,   4,   0],
       [  0,  54,   1,   0,   0,   0,   0],
       [  0,  13, 364,   0,   3,   2,   9],
       [  2, 461, 176,  10,   8,   6,  10],
       [  0, 151,   4,   0,   2,   1,   0],
       [  0,   3,   0,   0,   0,  69,   0],
       [  0, 170,   9,   0,   0,   1,  10]], dtype=int64)

In [59]:
metrics.accuracy_score(steel1['Fault'],steel1_final_pred)

0.26275115919629055

In [61]:
steel1.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Fault,TypeOfSteel
0,42,50,270900,270944,267,17,44,24220,76,108,...,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,4,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,4,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,4,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,4,1
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,4,1


In [62]:
l3=[]
l4=[]
for i in range(len(steel1)):
    if steel1['TypeOfSteel_A3FalseFalse'][i]:
        l3.append(1)
        l4.append(0)
    else:
        l3.append(0)
        l4.append(1)

In [63]:
steel1['TypeOfSteel_A3FalseFalse']=l3
steel1['TypeOfSteel_A4FalseFalse']=l4

In [64]:
steel1['TypeOfSteel_A3FalseFalse']=steel1['TypeOfSteel_A3FalseFalse'].astype('int')
steel1['TypeOfSteel_A4FalseFalse']=steel1['TypeOfSteel_A4FalseFalse'].astype('int')

In [72]:
steel1_num_normalized1 = preprocessing.scale(X = steel1.drop(['X_Minimum','X_Perimeter','Y_Perimeter','Sum_of_Luminosity','Log_X_Index','Log_Y_Index','SigmoidOfAreas','TypeOfSteel','Fault'],axis=1))

In [73]:
df_steel1_num_normalized1=pd.DataFrame(steel1_num_normalized1)

In [74]:
nb_classifier = naive_bayes.GaussianNB()
nb_classifier.fit(X = df_steel1_num_normalized1, y = steel1['Fault'])

GaussianNB(priors=None)

In [75]:
num_pred1=nb_classifier.predict(X=df_steel1_num_normalized1)

In [76]:
metrics.confusion_matrix(steel1['Fault'],num_pred1)

array([[264,  14,   6,  32,  70,   3,  13],
       [  6,  40,   0,   3,   6,   0,   0],
       [  3,   0, 349,  32,   5,   2,   0],
       [175,  46,  68, 193, 159,   8,  24],
       [ 15,   5,   0,   5, 124,   0,   9],
       [  1,   0,   0,   0,   1,  70,   0],
       [  7,   1,   8,   7,   8,   0, 159]], dtype=int64)

In [77]:
metrics.accuracy_score(steel1['Fault'],num_pred1)

0.61772282328696548

In [90]:
cohen_kappa_score(num_pred1,steel1['Fault'])

0.53792712694008604

In [79]:
df_steel1_num_normalized1.cov()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000515,0.052174,0.052162,-0.225515,0.168736,-0.062424,0.299545,0.112067,-0.112067,0.106174,0.242972,-0.152759,0.048600,-0.215041,0.149336,0.272055,0.099304,-0.332340,0.115079,-0.039016
1,0.052174,1.000515,1.000515,0.017679,-0.065736,-0.067820,-0.049236,0.075202,-0.075202,-0.207747,0.021325,-0.043139,-0.006138,0.054192,0.066119,-0.036562,-0.062944,0.044975,-0.086541,-0.090700
2,0.052162,1.000515,1.000515,0.017850,-0.065767,-0.067811,-0.049245,0.075189,-0.075189,-0.207751,0.021311,-0.043108,-0.006155,0.054213,0.066085,-0.036568,-0.062934,0.045017,-0.086524,-0.090713
3,-0.225515,0.017679,0.017850,1.000515,-0.497461,0.110119,-0.155933,-0.235713,0.235713,-0.183830,-0.275430,0.272949,0.017875,0.588909,-0.294825,-0.463810,-0.109711,0.650569,-0.137675,-0.043471
4,0.168736,-0.065736,-0.065767,-0.497461,1.000515,0.429826,-0.023591,0.042070,-0.042070,0.103446,0.359100,-0.044134,0.066782,-0.487825,0.252386,0.316773,0.035480,-0.679112,0.057152,0.669879
5,-0.062424,-0.067820,-0.067811,0.110119,0.429826,1.000515,-0.098059,-0.216450,0.216450,-0.128463,0.149752,0.031441,0.065551,0.099351,0.093570,-0.167527,-0.124102,0.007676,-0.169835,0.870609
6,0.299545,-0.049236,-0.049245,-0.155933,-0.023591,-0.098059,1.000515,0.378738,-0.378738,0.214880,0.135222,-0.230720,0.073732,-0.217529,0.123649,0.235853,0.128730,-0.193347,0.120778,-0.149846
7,0.112067,0.075202,0.075189,-0.235713,0.042070,-0.216450,0.378738,1.000515,-1.000515,0.125714,0.112198,-0.092001,0.164241,-0.244892,0.173926,0.240758,0.022154,-0.329784,0.010635,-0.252949
8,-0.112067,-0.075202,-0.075189,0.235713,-0.042070,0.216450,-0.378738,-1.000515,1.000515,-0.125714,-0.112198,0.092001,-0.164241,0.244892,-0.173926,-0.240758,-0.022154,0.329784,-0.010635,0.252949
9,0.106174,-0.207747,-0.207751,-0.183830,0.103446,-0.128463,0.214880,0.125714,-0.125714,1.000515,0.063482,0.012532,-0.124446,-0.228469,-0.077448,0.252115,0.221358,-0.176730,0.274238,-0.116559


# Decision Tree

In [80]:
from sklearn.tree import DecisionTreeClassifier

In [96]:
model_tree = DecisionTreeClassifier(criterion='entropy',max_depth=6)

In [97]:
model_tree.fit(X=df_steel1_num_normalized1,y=steel1['Fault'])

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [98]:
pred_tree_training = model_tree.predict(X =df_steel1_num_normalized1)

In [99]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score, accuracy_score
print(confusion_matrix(steel1['Fault'],pred_tree_training))
print(cohen_kappa_score(pred_tree_training,steel1['Fault']))
print(accuracy_score(steel1['Fault'],pred_tree_training))

[[294   1   3  95   6   0   3]
 [  3  32   0  19   1   0   0]
 [  7   0 377   5   2   0   0]
 [152   0  55 411  39   0  16]
 [ 15   1   0  69  73   0   0]
 [  2   0   0   5   0  65   0]
 [  5   0   8  10   2   0 165]]
0.652628839466
0.730036063885


# Grid Search

In [100]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [101]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(df_steel1_num_normalized1,steel1['Fault'], 
                                                    test_size=0.4, random_state=12345)

In [102]:
grid_param = {'criterion':['gini','entropy'], 'max_depth':[3,4,6,7]}
final_tree = GridSearchCV(model_tree, param_grid=grid_param,cv=5,n_jobs=-1, scoring = 'accuracy')
final_tree.fit(X=Train_X, y=Train_Y)
cross_val_score(final_tree, X=Train_X, y=Train_Y,cv=5).mean()

0.71304520645072722

In [103]:
final_tree.best_params_

{'criterion': 'gini', 'max_depth': 7}

In [104]:
pred_tree = final_tree.predict(X=Test_X)
print(confusion_matrix(pred_tree, Test_Y))
print(accuracy_score(pred_tree, Test_Y))

[[ 97   1   4  55   8   2   2]
 [  3  15   0   3   1   0   0]
 [  0   0 140   4   0   1   0]
 [ 51   8  13 173  35   3   4]
 [ 11   1   0  21  20   0   4]
 [  0   0   1   0   0  22   0]
 [  1   0   0   8   1   0  64]]
0.683397683398


In [105]:
pred_prob = final_tree.predict_proba(X = Test_X)

In [106]:
Pred_0 = pred_prob[:,0]

In [107]:
D = pd.DataFrame([Pred_0, Test_Y]).T
D.shape

(777, 2)